In [5]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
import torch
from datasets import load_dataset

In [2]:
model_checkpoint = "KBLab/bert-base-swedish-cased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at KBLab/bert-base-swedish-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/399k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
#datasest
data_files = {"train": "swerick_data_train.pkl", "test": "swerick_data_test.pkl"}
swerick_dataset = load_dataset("pandas",data_files=data_files)
print(swerick_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['protocole', 'texte', '__index_level_0__'],
        num_rows: 104
    })
    test: Dataset({
        features: ['protocole', 'texte', '__index_level_0__'],
        num_rows: 26
    })
})


In [15]:
sample = swerick_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> protocle: {row['protocole']}'")
    print(f"'>>> Texte: {row['texte']}'")
    print(f"'>>> index: {row['__index_level_0__']}'")


'>>> protocle: 70'
'>>> Texte:           RIKSDAGENS                  1956 ANDRA KAMMAREN Nr 7                  17—22 februari                  Debatter m. m.                  Fredagen den 17 februari                  Sid. Interpellation av herr Stenberg ang. möjligheterna att stanna          snabbgående tåg i händelse av stopptecken vid bevakade järnvägsöver-          GÅDDAL 1ooms ejer a oyje soda: om 1 16 IRS RE BIR SR BENA FAT          SRA RR: 3 Tisdagen den 21 februari Interpellationer av: fru Sjöstrand          ang. rätt för dem, som förtidspensionerats på grund av sjukdom          och invaliditet att utnyttja billighetsresorna för folk- PENSlONÄLEI          >. send sie Ed GKK kr MR cs RR sf VR SN 6 herr Hagård ang. de          nuvarande tiderna för rusdrycksförsäljning. . 7 herr Edström ang.          vissa menliga följder för aktiebolag och ekonomiska föreningar,          som på grund av missuppfattning kommit att erlägga för lågt skattebelopp          i samband med den år 1955 g

In [12]:
def tokenize_function(examples):
    result = tokenizer(examples["texte"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [16]:
tokenized_datasets = swerick_dataset.map(
    tokenize_function, batched=True, remove_columns=["texte", "protocole",'__index_level_0__']
)
tokenized_datasets

Map:   0%|          | 0/104 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 104
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 26
    })
})

In [17]:
tokenizer.model_max_length

1000000000000000019884624838656

In [19]:
chunk_size = 1000000

In [20]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 15034'
'>>> Review 1 length: 141741'
'>>> Review 2 length: 103737'
